# Electricity Load Forecasting (h+1) — Baseline & First Machine Learning Model

The goal of this notebook is to build a first **machine learning model** to predict **hourly electricity demand one hour ahead (h+1)** using historical load, weather, and calendar features.

This notebook follows **time-series best practices**:
- No data leakage
- Time-aware train/validation/test splits
- Baseline comparison
- Progressive model complexity

## 1. Problem formulation

We formulate the task as a **supervised regression problem** with:
* **Features (X)** available at time t:
    * Load at time t
    * Load lags: t-1, t-24, t-168
    * Temperature at time t
    * Calendar features (hour, weekday, week of year)
* **Target (y)**:
    * Load at time t+1
    
Mathematically:
$$\hat{y}_{t+1} = f(X_t)$$

## 2. Imports & Setup

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

## 3. Project Paths and Parameters

In [2]:
PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_BASE_PATH = PROJECT_ROOT / "data" / "processed"

In [3]:
# Parameters
country = "FR"
years = list(range(2015, 2024+1))

## 4. Load feature-engineered dataset

The dataset consists of **hourly electricity demand and weather data** for France between 2015 and 2024.

Key characteristics:
* Hourly resolution
* Strong daily, weekly and annual seasonality
* High temporal autocorrelation
* Strong dependency on temperature

In [4]:
# Load feature-engineered data
dfs = []
for year in years:
    path = (
        PROCESSED_BASE_PATH
        / f"country={country}"
        / f"year={year}"
        / "load_forecasting_features.parquet"
    )
    dfs.append(pd.read_parquet(path))

df = pd.concat(dfs).sort_index()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87502 entries, 0 to 8783
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   datetime         87502 non-null  datetime64[ns, UTC]
 1   load_t           87502 non-null  float64            
 2   load_t-1         87502 non-null  float64            
 3   load_t-24        87502 non-null  float64            
 4   load_t-168       87502 non-null  float64            
 5   temperature_t    87502 non-null  float32            
 6   hour             87502 non-null  int32              
 7   is_weekday       87502 non-null  int64              
 8   week_of_year     87502 non-null  int64              
 9   target_load_t+1  87502 non-null  float64            
dtypes: datetime64[ns, UTC](1), float32(1), float64(5), int32(1), int64(2)
memory usage: 6.7 MB


In [6]:
df.head()

,datetime,load_t,load_t-1,load_t-24,load_t-168,temperature_t,hour,is_weekday,week_of_year,target_load_t+1
0,2015-01-08 00:00:00+00:00,65948.0,68372.0,67621.0,70929.0,5.154,0,1,2,64676.0
0,2017-01-01 00:00:00+00:00,73330.0,75140.0,70296.0,57480.0,-3.600,0,0,52,71806.0
0,2018-01-01 00:00:00+00:00,56036.0,57638.0,52993.0,58872.0,8.100,0,0,1,54494.0
0,2019-01-01 00:00:00+00:00,60301.0,62176.0,59352.0,54739.0,6.100,0,0,1,58540.0
0,2020-01-01 00:00:00+00:00,63142.0,64677.0,63434.0,51806.0,2.350,0,0,1,61612.0


## 5. Feature matrix and target

In [7]:
# Transform datetime to index
df = df.assign(datetime=df["datetime"]).set_index("datetime").sort_index()

In [8]:
# Check that the index is sorted
assert df.index.is_monotonic_increasing

In [9]:
# Target column
target_col = "target_load_t+1"

# Features columns
feature_cols = [
    "load_t",
    "load_t-1",
    "load_t-24",
    "load_t-168",
    "temperature_t", # For the moment, we only use temperature from weather data
    "hour",
    "is_weekday",
    "week_of_year",
]

X = df[feature_cols]
y = df[target_col]

## 5. Time-based data split

In time series forecasting, **random splits are invalid** because future information would leak into the past.

We use a chronological split:
* Train: 2015–2022
* Validation: 2023
* Test: 2024

In [10]:
# Split dates : train (2015-2022), val (2023), test (2024)
train_end = pd.Timestamp("2022-12-31 23:00:00", tz="UTC")
val_end = pd.Timestamp("2023-12-31 23:00:00", tz="UTC")

X_train = X.loc[:train_end]
y_train = y.loc[:train_end]

X_val = X.loc[train_end:val_end]
y_val = y.loc[train_end:val_end]

X_test = X.loc[val_end:]
y_test = y.loc[val_end:]

print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)

Train: (69959, 8)
Validation: (8761, 8)
Test: (8784, 8)


## 6. Baseline model

The **baseline model** is a persistence model:
$$\hat{y}_{t+1} = y_t$$

It assumes that the next hour's load equals the current load. The baseline model is the simplest possible forecasting strategy used as a **minimum performance threshold**.

In [11]:
# Baseline model: predicting load_t+1 as load_t
y_val_pred_baseline = X_val["load_t"]

In [26]:
# Evaluation
baseline_mae = mean_absolute_error(y_val, y_val_pred_baseline)
baseline_rmse = mean_squared_error(y_val, y_val_pred_baseline)

print(f"Baseline MAE:  {baseline_mae:.2f}")
print(f"Baseline RMSE: {baseline_rmse:.2f}")

Baseline MAE:  1560.09
Baseline RMSE: 3768238.37


Results:
* MAE ≈ 1560 MW
* Confirms strong short-term persistence

## 7. Feature scaling

**Why scaling is required?**

Ridge regression applies an L2 penalty on coefficients:
$$\alpha \sum_{j=1}^{p} \beta_j^2$$

Without scaling:
* Load features dominate
* Calendar and temperature effects are suppressed

We therefore **standardize all input features**.

## 8. Ridge Regression

Ridge regression is used as a **first machine learning benchmark** because it naturally handles correlated lag features, reduces variance through regularization, and provides a strong yet interpretable baseline before moving to more complex non-linear models.

Ridge regression minimizes:
$$\min_{\beta} \left( \sum_{i=1}^{n}(y_i - X_i\beta)^2 + \alpha \sum_{j=1}^{p}\beta_j^2 \right)$$

In [18]:
# Ridge Pipeline
ridge_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()), # mandatory to scale features for linear models such as ridge
        ("model", Ridge(alpha=0.01)) # obtained after hyperparameter tuning
    ]
)

ridge_pipeline.fit(X_train, y_train)
y_val_pred = ridge_pipeline.predict(X_val)

In [19]:
# Evaluation
ridge_mae = mean_absolute_error(y_val, y_val_pred)
ridge_rmse = mean_squared_error(y_val, y_val_pred)

print(f"Ridge MAE:  {ridge_mae:.2f}")
print(f"Ridge RMSE: {ridge_rmse:.2f}")

Ridge MAE:  1110.29
Ridge RMSE: 2137279.95


**Interpretation:**

* **Mean Absolute Error**: 

On average, the model’s hourly forecast is off by about 1.1 GW.

Given that French hourly electricity demand typically ranges between 30 and 90 GW, this corresponds to an average relative error of roughly 1–4%, which is very reasonable for short-term load forecasting.

* **Root Mean Squared Error**: 

RMSE penalizes large errors more strongly than MAE.

The fact that RMSE is significantly higher than MAE indicates that:
- Most predictions are reasonably accurate.
- A small number of hours (typically winter peaks or sudden demand changes) produce larger errors.

This behavior is expected in electricity demand forecasting, especially during extreme weather events.

## 8. Hyperparameter tuning (alpha)

Idea behind the algorithm:
1. Define a grid of α values (log scale)
2. For each α:
    - Train Ridge on past data
    - Validate on future data
3. Select α with lowest validation error
4. Never use test data for tuning

This ensures:
- No temporal leakage
- Robust hyperparameter selection

In [ ]:
# Hyperparameter tuning for Ridge regression using TimeSeriesSplit CV
alphas = np.logspace(-2, 3, 10)  # 0.01 → 1000

tscv = TimeSeriesSplit(n_splits=5) # time series cross-validation

best_alpha = None
best_mae = np.inf

# Iterate over alphas
for alpha in alphas:
    maes = []

    for train_idx, val_idx in tscv.split(X_train):
        X_tr, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]

        pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("model", Ridge(alpha=alpha))
        ])

        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict(X_val_cv)
        maes.append(mean_absolute_error(y_val_cv, y_pred))

    avg_mae = np.mean(maes) # average MAE for current alpha

    # Check for best alpha
    if avg_mae < best_mae:
        best_mae = avg_mae
        best_alpha = alpha

print(f"Best alpha: {best_alpha}")
print(f"CV MAE: {best_mae:.2f}")

Best alpha: 0.01
CV MAE: 1184.66


## 9. Models comparison

In [ ]:
# DataFrame summarizing results
pd.DataFrame({
    "Model": ["Baseline (load_t)", "Ridge Regression"],
    "MAE": [baseline_mae, ridge_mae],
    "RMSE": [baseline_rmse, ridge_rmse],
})

,Model,MAE,RMSE
0,Baseline (load_t),1560.091428,3.768238e+06
1,Ridge Regression,1110.291551,2.137280e+06


Interpretation: 
- Ridge reduces MAE by ~29%
- Ridge reduces RMSE by ~43%

This clearly shows that:
- Simple persistence captures temporal inertia
- Weather + calendar information adds substantial predictive power
- Linear models already outperform naive heuristics by a large margin

This confirms the value of feature engineering and the usefulness of weather and calendar features.

## 10. Model interpretation

In [27]:
coef = ridge_pipeline.named_steps["model"].coef_
feature_importance = pd.Series(coef, index=feature_cols).sort_values(ascending=False)

print(feature_importance)

load_t           17697.684518
is_weekday         198.160937
load_t-24          174.132144
week_of_year       -55.686737
load_t-168         -70.682740
hour              -286.288508
temperature_t     -388.766126
load_t-1         -6835.006385
dtype: float64


All features were standardized before training, so coefficients are directly comparable in magnitude.

The Ridge model is dominated by strong load inertia: 
- The **current load (load_t)** is by far the **most influential predictor**, confirming high short-term persistence in electricity demand. 
- **Temperature** is the main exogenous driver, with colder conditions increasing demand. 
- **Lagged daily and weekly loads** capture residual seasonality, while **calendar variables** (hour, weekday) refine intra-day and behavioral patterns. 

Overall, the coefficients align well with physical and temporal intuition and validate the feature engineering choices.

## 11. Final evaluation on test set

In [28]:
y_test_pred = ridge_pipeline.predict(X_test)

test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred)

print(f"Ridge MAE:  {test_mae:.2f}")
print(f"Ridge RMSE: {test_rmse:.2f}")

Ridge MAE:  1096.86
Ridge RMSE: 2042324.64


Key insights:
- Test performance is very close to validation
- Model generalizes well to unseen future data

This is a strong signal of model stability, especially in time series forecasting.

## 12. Conclusion

In this notebook, we developed and evaluated a first machine learning model for short-term electricity demand forecasting (h+1).

Exploratory analysis revealed strong **multi-scale seasonality**, high temporal persistence, and a clear **dependency on temperature**.** Based on these insights, we engineered lagged load, weather, and calendar features and formulated the problem as a supervised regression task.

A **persistence baseline** highlighted the **strong inertia of electricity demand**. **Ridge regression significantly improved forecasting accuracy** by combining temporal information with exogenous variables while controlling for multicollinearity through L2 regularization.

The model achieves **stable performance** across validation and test sets, with an average error of approximately 1.1 GW, corresponding to a low relative error at the national scale. Coefficient analysis confirms that current load level is the dominant predictor, with temperature as the main external driver, and calendar features refining intra-day and seasonal patterns.

Overall, this model provides a strong, interpretable baseline for operational forecasting and establishes a solid foundation for more advanced approaches such as gradient boosting models, longer forecast horizons (h+24), and production deployment.